In [ ]:
import warnings

warnings.filterwarnings(action="ignore")

# Data from POH of SW121

In [ ]:
import pandas as pd
import numpy as np

altitude_validation_points = np.array(
    [
        2000.0,
        2000.0,
        2000.0,
        2000.0,
        2000.0,
        4000.0,
        4000.0,
        4000.0,
        4000.0,
        6000.0,
        6000.0,
        6000.0,
        8000.0,
        8000.0,
        8000.0,
        10000.0,
        10000.0,
        12000.0,
    ]
)
rpm_validation_points = np.array(
    [
        5500.0,
        5500.0,
        5300.0,
        4900.0,
        4600.0,
        5500.0,
        5500.0,
        5100.0,
        4600.0,
        5500.0,
        5300.0,
        4900.0,
        5500.0,
        5300.0,
        5100.0,
        5300.0,
        5500.0,
        5500.0,
    ]
)
power_validation_points = (
    np.array(
        [
            1.0,
            0.85,
            0.75,
            0.65,
            0.55,
            0.85,
            0.75,
            0.65,
            0.55,
            0.75,
            0.65,
            0.55,
            0.75,
            0.65,
            0.55,
            0.65,
            0.55,
            0.55,
        ]
    )
    * 69.0
)  # Power is given as percentage of MCP
map_validation_points = np.array(
    [
        27.7,
        26.7,
        25.7,
        24.7,
        24.0,
        25.3,
        24.3,
        23.3,
        23.3,
        23.3,
        22.7,
        22.0,
        22.0,
        21.7,
        21.0,
        19.7,
        20.3,
        18.0,
    ]
)  # In inHg
vol_ff_validation_points = np.array(
    [
        28.8,
        22.4,
        18.4,
        16.0,
        14.4,
        25.2,
        19.6,
        16.8,
        15.6,
        23.2,
        19.6,
        16.8,
        23.6,
        21.2,
        18.0,
        22.4,
        19.2,
        20.4,
    ]
)  # In l/h

displacement_volume = 1352.0  # In cm**3
displacement_volume /= 1e6

mep_validation_points = (power_validation_points * 2.0 * np.pi * 4.0) / (
    displacement_volume * rpm_validation_points * 2.0 * np.pi / 60.0
)

# Regression for sfc

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

import pandas as pd

from stdatm import Atmosphere

sigma_validation_points = (
    Atmosphere(altitude_validation_points, altitude_in_feet=True).density / Atmosphere(0).density
)
rpm_ratio_validation_points = rpm_validation_points / 5500.0

# Multiplicative factor to sfc at max rpm sea level
sfc_validation_point = (
    vol_ff_validation_points * 0.72 / power_validation_points * 1000.0
)  # in g/kWh
k_sfc_validation_points = sfc_validation_point / 285.0

pi1 = sigma_validation_points
pi2 = map_validation_points / 29.92  # SL pressure in inHg
# pi2 = mep_validation_points / 29.92  # SL pressure in inHg
pi3 = k_sfc_validation_points
doe_sfc = np.c_[pi1, pi2, pi3]
doe_sfc = pd.DataFrame(doe_sfc, columns=["pi1", "pi2", "pi3"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_sfc.values, elected_pi0="pi3", order=3, log_space=False, test_mode=True
)
perform_regression(doe_sfc.values, model, chosen_model=4)

In [ ]:
print(k_sfc_validation_points)

# Regression to link power rate to MAP

Power rate seems like a better fit to estimate the MAP but that means that we must be vigilant that the polynomial doesn't diverge when the power rate goes above 1 (which might happen in eatly loops)

In [ ]:
pi1 = sigma_validation_points
pi2 = power_validation_points / 69.0
pi3 = map_validation_points / 29.92  # SL pressure in inHg

doe_map = np.c_[pi1, pi2, pi3]
doe_map = pd.DataFrame(doe_map, columns=["pi1", "pi2", "pi3"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_map.values, elected_pi0="pi3", order=3, log_space=False, test_mode=True
)
perform_regression(doe_map.values, model, chosen_model=4)

In [ ]:
def manifold_pressure(density_ratio, power_rate):
    """Returns the MAP in inHg."""
    map_ratio = (
        -0.25089
        + 0.30569 * density_ratio**2.0 * power_rate
        + 1.66572 * density_ratio
        - 0.74920 * density_ratio**2.0
    )

    return map_ratio * 29.92


def sfc(density_ratio, manifold_pressure):
    """Returns the sfc in g/kWh."""

    sfc_ratio = (
        0.06437
        + 6.92209 * density_ratio
        + 5.17683 * manifold_pressure**3.0
        - 11.05164 * density_ratio * manifold_pressure
    )

    return sfc_ratio * 285.0


def r2(ref_values, interp_values):
    return 1.0 - np.sum((ref_values - interp_values) ** 2) / np.sum(
        (ref_values - np.mean(ref_values)) ** 2
    )

In [ ]:
map_cross_ref = manifold_pressure(sigma_validation_points, power_validation_points / 69.0)
print(map_cross_ref)
print(map_validation_points)

sfc_cross_ref = sfc(sigma_validation_points, map_cross_ref / 29.92)
print(sfc_cross_ref)
print(sfc_validation_point)

vol_ff_cross_ref = sfc_cross_ref * power_validation_points / 1000.0 / 0.72

print(vol_ff_cross_ref)
print(vol_ff_validation_points)

# Verfication time

Now we will check the results on another aircraft using the same Rotax (Tecnam P2006T), and check another aircraft using another Rotax (like the 914 if possible). Actually the check on the Tecnam is not relevant because it seems like its SFC is constant regardless of conditions.

In [ ]:
p2006t_altitude = np.array([0.0, 0.0, 0.0, 3000.0, 3000.0, 3000.0, 6000.0, 6000.0, 9000.0])  # In ft
p2006t_power_ratio = np.array([0.83, 0.63, 0.50, 0.85, 0.65, 0.48, 0.54, 0.62, 0.65])  # In %
p2006t_map = np.array([28.0, 26.0, 24.0, 26.4, 24.0, 22.0, 20.0, 22.0, 21.1])  # In inHg
p2006t_ff = np.array([23.1, 17.5, 14.1, 23.6, 18.1, 13.3, 14.9, 17.2, 18.3])  # In l/h

p2006t_computed_sigma = (
    Atmosphere(p2006t_altitude, altitude_in_feet=True).density / Atmosphere(0).density
)

p2006t_computed_map = manifold_pressure(p2006t_computed_sigma, p2006t_power_ratio)
print("Computed MAP", p2006t_computed_map)
print("Actual MAP", p2006t_map)
print("R2 MAP", r2(p2006t_map, p2006t_computed_map))

p2006t_computed_sfc = sfc(p2006t_computed_sigma, p2006t_computed_map / 29.92)
p2006t_computed_ff = p2006t_computed_sfc * p2006t_power_ratio * 69.0 / 1000.0 / 0.72
print("Computed FF", p2006t_computed_ff)
print("Actual FF", p2006t_ff)
print("R2 MAP", r2(p2006t_ff, p2006t_computed_ff))